# Create ActiGraph counts from raw data

Since [ActiGraph has released their count-processing algorithm](https://github.com/actigraph/agcounts), it is now fairly easy to obtain ActiGraph/ActiLife counts even without using ActiLife. In this tutorial, we will show how to do this using PAAT.

In [1]:
%%capture
import paat

## Load the data

First, we have to load our example GT3X file again:

In [2]:
data, sample_freq = paat.read_gt3x("data/10min_recording.gt3x")
data

,X,Y,Z
2022-01-03 10:20:00.000,0.804688,0.621094,0.085938
2022-01-03 10:20:00.010,0.804688,0.597656,0.085938
2022-01-03 10:20:00.020,0.804688,0.585938,0.078125
2022-01-03 10:20:00.030,0.804688,0.582031,0.074219
2022-01-03 10:20:00.040,0.800781,0.585938,0.074219
...,...,...,...
2022-01-03 10:29:59.950,0.289062,0.960938,-0.050781
2022-01-03 10:29:59.960,0.289062,0.960938,-0.054688
2022-01-03 10:29:59.970,0.285156,0.957031,-0.054688
2022-01-03 10:29:59.980,0.289062,0.957031,-0.054688


## Create ActiGraph counts over arbitrary epoch lengths

PAAT has implemented a `calculate_actigraph_counts()` functions which requires the `data`, the `sample_freq` and a string indicating the epoch length (e.g. `"1s"` for one second epochs, `"10s` for ten second epochs or `"1min"` for one minute epochs). In the backend, PAAT is passing most of the processing to [ActiGraph's agcounts](https://github.com/actigraph/agcounts) which could also be used independently. However, we implemented a convenience wrapper that allows passing the epoch length as a string (as commonly done with pandas) and which correctly updates the DataFrame's index with the correct time stamps. 

Here, for example is the calculation of one second epochs shown:

In [3]:
counts_1sec = paat.calculate_actigraph_counts(data, sample_freq, "1s")
counts_1sec.head()

,Y,X,Z
2022-01-03 10:20:00,0,0,0
2022-01-03 10:20:01,0,0,0
2022-01-03 10:20:02,0,0,0
2022-01-03 10:20:03,0,0,0
2022-01-03 10:20:04,0,0,0


To calculate ten second epochs, one does only need to adjust the `epoch_length` argument.

In [4]:
counts_10sec = paat.calculate_actigraph_counts(data, sample_freq, "10s")
counts_10sec.head()

,Y,X,Z
2022-01-03 10:20:00,0,0,0
2022-01-03 10:20:10,0,0,0
2022-01-03 10:20:20,0,0,0
2022-01-03 10:20:30,0,0,0
2022-01-03 10:20:40,185,376,204


If you created a one second epoch DataFrame, it is also no problem to aggregate it to arbitrary epochs later using [Pandas' resample function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html):

In [5]:
new_counts_10sec = counts_1sec.resample("10s").sum()
new_counts_10sec.head()

,Y,X,Z
2022-01-03 10:20:00,0,0,0
2022-01-03 10:20:10,0,0,0
2022-01-03 10:20:20,0,0,0
2022-01-03 10:20:30,0,0,0
2022-01-03 10:20:40,185,376,204


## Create Brønd counts

Alternatively, you can use the open source algorithm proposed by [Brønd et al. (2017)](http://dx.doi.org/10.1249/MSS.0000000000001344):

In [6]:
brond_counts = paat.calculate_brond_counts(data, sample_freq, "10s")
brond_counts.head()

,Y,X,Z
2022-01-03 10:20:00,28,35,0
2022-01-03 10:20:10,0,0,0
2022-01-03 10:20:20,0,0,0
2022-01-03 10:20:30,0,0,0
2022-01-03 10:20:40,188,380,205


An alternative implementation can be found [here](https://github.com/jbrond/ActigraphCounts).